# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments        Customer  \
0  Jan  3 2023 12:55PM  254180        17  WT5504 Out  Citieffe, Inc.   
1  Jan  3 2023 12:24PM  254178        12    HH-39141       Hush Hush   
2  Jan  3 2023 12:24PM  254178        12    HH-39142       Hush Hush   
3  Jan  3 2023 12:24PM  254178        12    HH-39143       Hush Hush   
4  Jan  3 2023 12:24PM  254178        12    HH-39144       Hush Hush   
5  Jan  3 2023 12:24PM  254178        12    HH-39145       Hush Hush   
6  Jan  3 2023 12:24PM  254178        12    HH-39146       Hush Hush   
7  Jan  3 2023 12:24PM  254178        12    HH-39147       Hush Hush   
8  Jan  3 2023 12:24PM  254178        12    HH-39148       Hush Hush   
9  Jan  3 2023 12:24PM  254178        12    HH-39149       Hush Hush   

  ShipmentStatus  
0      Executing  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
49  254172      Executing          3
50  254174       Released          1
51  254177       Released          2
52  254178       Released         20
53  254180      Executing          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
254172                NaN        3.0       NaN
254174                NaN        NaN       1.0
254177                NaN        NaN       2.0
254178                NaN        NaN      20.0
254180                NaN        1.0       NaN

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254090               37.0       20.0       5.0
254104                0.0        0.0       1.0
254108                0.0        1.0      23.0
254111                0.0        0.0      53.0
254117                0.0        0.0      37.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
254090                 37         20         5
254104                  0          0         1
254108                  0          1        23
254111                  0          0        53
254117                  0          0        37

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               254090         37         20         5
1               254104          0          0         1
2               254108          0          1        23
3               254111          0          0        53
4               254117          0          0        37

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               254090        37        20        5
1               254104                            1
2               254108                   1       23
3               254111                           53
4               254117                           37

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse               Customer
0    Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.
1    Jan  3 2023 12:24PM  254178        12              Hush Hush
21   Jan  3 2023 12:06PM  254177        19  ACG North America LLC
23   Jan  3 2023 12:01PM  254174        15                  Lonza
24   Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.
136  Jan  3 2023 11:51AM  254172        19  VITABIOTICS  USA INC.
139  Jan  3 2023 11:51AM  254157        15  Alliance Pharma, Inc.
141  Jan  3 2023 11:46AM  254170        20         Exact-Rx, Inc.
142  Jan  3 2023 11:45AM  254169        20         Exact-Rx, Inc.
143  Jan  3 2023 11:43AM  254167        20         Exact-Rx, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse               Customer Completed  \
0  Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.             
1  Jan  3 2023 12:24PM  254178        12              Hush Hush             
2  Jan  3 2023 12:06PM  254177        19  ACG North America LLC             
3  Jan  3 2023 12:01PM  254174        15                  Lonza             
4  Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.             
5  Jan  3 2023 11:51AM  254172        19  VITABIOTICS  USA INC.             
6  Jan  3 2023 11:51AM  254157        15  Alliance Pharma, Inc.             
7  Jan  3 2023 11:46AM  254170        20         Exact-Rx, Inc.             
8  Jan  3 2023 11:45AM  254169        20         Exact-Rx, Inc.             
9  Jan  3 2023 11:43AM  254167        20         Exact-Rx, Inc.             

  Executing Released  
0         1           
1                 20  
2                  2  
3                  1  
4                112  
5         3           
6                  2  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse               Customer Released  \
0  Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.            
1  Jan  3 2023 12:24PM  254178        12              Hush Hush       20   
2  Jan  3 2023 12:06PM  254177        19  ACG North America LLC        2   
3  Jan  3 2023 12:01PM  254174        15                  Lonza        1   
4  Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.      112   
5  Jan  3 2023 11:51AM  254172        19  VITABIOTICS  USA INC.            
6  Jan  3 2023 11:51AM  254157        15  Alliance Pharma, Inc.        2   
7  Jan  3 2023 11:46AM  254170        20         Exact-Rx, Inc.        1   
8  Jan  3 2023 11:45AM  254169        20         Exact-Rx, Inc.        1   
9  Jan  3 2023 11:43AM  254167        20         Exact-Rx, Inc.        1   

  Executing Completed  
0         1            
1                      
2                      
3                      
4                      
5         3            
6                      
7                      
8                      
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse               Customer Released  \
0  Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.            
1  Jan  3 2023 12:24PM  254178        12              Hush Hush       20   
2  Jan  3 2023 12:06PM  254177        19  ACG North America LLC        2   
3  Jan  3 2023 12:01PM  254174        15                  Lonza        1   
4  Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.      112   

  Executing Completed  
0         1            
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.       NaN   
1  Jan  3 2023 12:24PM  254178        12              Hush Hush      20.0   
2  Jan  3 2023 12:06PM  254177        19  ACG North America LLC       2.0   
3  Jan  3 2023 12:01PM  254174        15                  Lonza       1.0   
4  Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.     112.0   

   Executing  Completed  
0        1.0        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse               Customer  Released  \
0  Jan  3 2023 12:55PM  254180        17         Citieffe, Inc.       0.0   
1  Jan  3 2023 12:24PM  254178        12              Hush Hush      20.0   
2  Jan  3 2023 12:06PM  254177        19  ACG North America LLC       2.0   
3  Jan  3 2023 12:01PM  254174        15                  Lonza       1.0   
4  Jan  3 2023 11:52AM  254171        15   Person & Covey, Inc.     112.0   

   Executing  Completed  
0        1.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         6099294     718.0       35.0        0.0
12          762427      21.0        1.0        0.0
15         1270755     119.0       45.0       31.0
17          254180       0.0        1.0        0.0
19         1270711       8.0       26.0       37.0
20         1779090       7.0        0.0        0.0
21          254144       1.0        0.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  6099294     718.0       35.0        0.0
1        12   762427      21.0        1.0        0.0
2        15  1270755     119.0       45.0       31.0
3        17   254180       0.0        1.0        0.0
4        19  1270711       8.0       26.0       37.0
5        20  1779090       7.0        0.0        0.0
6        21   254144       1.0        0.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     718.0       35.0        0.0
1        12      21.0        1.0        0.0
2        15     119.0       45.0       31.0
3        17       0.0        1.0        0.0
4        19       8.0       26.0       37.0
5        20       7.0        0.0        0.0
6        21       1.0        0.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  718.0
1        12  Released   21.0
2        15  Released  119.0
3        17  Released    0.0
4        19  Released    8.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    12     15   17    19   20   21
Status                                            
Completed    0.0   0.0   31.0  0.0  37.0  0.0  0.0
Executing   35.0   1.0   45.0  1.0  26.0  0.0  0.0
Released   718.0  21.0  119.0  0.0   8.0  7.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    12     15   17    19   20   21
0          Completed    0.0   0.0   31.0  0.0  37.0  0.0  0.0
1          Executing   35.0   1.0   45.0  1.0  26.0  0.0  0.0
2           Released  718.0  21.0  119.0  0.0   8.0  7.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    12     15   17    19   20   21
0  Completed    0.0   0.0   31.0  0.0  37.0  0.0  0.0
1  Executing   35.0   1.0   45.0  1.0  26.0  0.0  0.0
2   Released  718.0  21.0  119.0  0.0   8.0  7.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()